In [ ]:
from openai import OpenAI
import json

import os
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ.get("OPENAI_API_KEY")
)

In [ ]:
system_prompt = """You are an expert career coach and cover letter writer.
Given a job description and candidate info, write a compelling, professional cover letter.

Rules:
- Keep it under 350 words
- Be specific to the job, not generic
- Highlight relevant skills that match the job requirements
- Use a confident but not arrogant tone
- Include a strong opening hook and a clear call to action at the end
- Do NOT use filler phrases like 'I am writing to express my interest'
"""

In [ ]:
def generate_cover_letter(job_description, candidate_info, model="gpt-4.1-nano"):
    """Generate a tailored cover letter with streaming output."""
    user_prompt = f"""Generate a cover letter for this job:

JOB DESCRIPTION:
{job_description}

CANDIDATE INFO:
{candidate_info}
"""
    stream = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        stream=True
    )
    result = ""
    for chunk in stream:
        content = chunk.choices[0].delta.content
        if content:
            print(content, end="")
            result += content
    return result

In [ ]:
# Example usage

job_desc = """
Senior Python Developer at TechCorp
- 3+ years Python experience
- Experience with REST APIs and microservices
- Knowledge of AWS or cloud platforms
- Familiarity with ML/AI tools is a plus
- Strong communication and teamwork skills
"""

candidate = """
Name: Mirack
Experience: 2 years as a software developer, 1 year focused on Python
Skills: Python, Flask, FastAPI, PostgreSQL, Docker, basic AWS (EC2, S3)
Currently learning: LLM engineering, AI/ML fundamentals
Education: BSc Computer Science
"""

cover_letter = generate_cover_letter(job_desc, candidate)

In [ ]:
def analyze_job_fit(job_description, candidate_info, model="gpt-4.1-nano"):
    """Analyze how well a candidate fits a job and return structured JSON."""
    prompt = f"""Analyze this job fit and respond ONLY with valid JSON, no markdown:

JOB: {job_description}
CANDIDATE: {candidate_info}

Return this exact JSON structure:
{{
    "match_score": "score out of 10",
    "matching_skills": ["list of skills that match"],
    "skill_gaps": ["list of missing skills"],
    "talking_points": ["3 key things to emphasize in interview"],
    "one_line_pitch": "a single sentence elevator pitch for this role"
}}
"""
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
analysis = analyze_job_fit(job_desc, candidate)

print(f"Match Score: {analysis['match_score']}")
print(f"\nMatching Skills: {', '.join(analysis['matching_skills'])}")
print(f"\nSkill Gaps: {', '.join(analysis['skill_gaps'])}")
print(f"\nTalking Points:")
for i, point in enumerate(analysis['talking_points'], 1):
    print(f"  {i}. {point}")
print(f"\nElevator Pitch: {analysis['one_line_pitch']}")